In [25]:
from typing import Tuple
import numpy as np
import pandas as pd

In [47]:
x = pd.DataFrame([[40, 230, 10], [20, 30, 40], [12, 30, 1]], columns=['a', 'b', 'c'])
y = pd.Series([[3], [2], [2]])
x.shape, y.shape

((3, 3), (3,))

In [40]:
class nn:
    def __init__(self, x, y, units):
        self.x, self.y, self.units = x, y, units
        self.w, self.b = None, None
        self.init_weight_bias(self.x, self.units)
        
    def linear(self):
        return self.x@self.w+self.b

    def sigmoid(self, x):
        return 1/np.exp(-x)

    def init_weight_bias(self, x, units):
        self.w = np.random.randn(units, x.shape[0])
        self.b = np.random.randn(1)
    

In [50]:
n = nn(np.array(x), np.array(y), 3)

In [51]:
n.linear()

array([[ 92.59520689, 364.06318999, 140.83467488],
       [ 13.65815163, 141.48143212, -56.97924281],
       [ 28.66541058,  51.80500164,  26.37650111]])